# Markov Reward Process

In [278]:
import numpy as np

In [279]:
SIMULATION = "simulation"
ANALYTIC = "analytic"
ITERATIVE = "iterative"

In [280]:
class MarkovRewardProcess():
    def __init__(self, alg=SIMULATION, gamma=0.99, n_episodes=100, epsilon=0.01):
        self.alg = alg
        self.gamma = gamma
        self.n_episodes = n_episodes
        self.epsilon = epsilon
        self.value_function = None
        self._time_steps = 500
    
    
    def fit(self, S, P, R):
        """
        S: Integer that represent the amount of states
        P: Transition probability matrix
        R: List of rewards at the length of len(S) (reward for each state)
        """
        # Set the value table
        self.value_function = np.zeros(S)
        
        if self.alg == SIMULATION: 
            self._fit_simulation(S, P, R)
        elif self.alg == ANALYTIC:
            self._fit_analytical_solution(S, P, R)
        else:
            self._fit_iterative_solution(S, P, R)


    
    def value_func(self):
        return np.asarray(self.value_function).reshape(1, -1)
    
    
    def _fit_simulation(self, S, P, R):
        for state in range(S):
            state_reward = 0
            for i in range(self.n_episodes):
                # generate episode and episode reward
                episode_reward = self._generate_episode(S, P, R, state)
                # add the reward to state_reward
                state_reward += episode_reward
            
            # normalize by n_episodes
            state_reward /= self.n_episodes
            # set v(s) = this value
            self.value_function[state] = state_reward
     
    
    def _fit_analytical_solution(self, S, P, R):
        # Identity matrix the size of n_states
        identity_matrix = np.identity(S)
        
        # Transform R to a vector
        R_vector = np.asarray(R).reshape(-1, 1)
        
        # Perform the analytical solution
        value_funct_vector = np.dot(np.linalg.inv(identity_matrix - self.gamma * P), R_vector)
        
        # Return the value function
        self.value_function = value_funct_vector.tolist()
    
    
    def _fit_iterative_solution(self, S, P, R):
        # Init the value function
        self.value_function = np.ones(S) * self.epsilon * 2
        temp_value_function = np.zeros(S)
        
        while np.linalg.norm(self.value_function - temp_value_function) > self.epsilon:
            self.value_function = temp_value_function.copy()
            for state in range(S):
                temp_value_function[state] = R[state] + self.gamma * np.dot(P[state], np.transpose(temp_value_function))

    
    def _generate_episode(self, S, P, R, state):
        t = 0
        episode_reward = 0
        while t < self._time_steps:
            # add reward from current state (with gamma!)
            episode_reward += np.power(self.gamma, t) * R[state]
            # move to a new state
            state = np.random.choice(S, p=P[state])
            # t++
            t += 1
        return episode_reward

In [281]:
S = 2
P = np.array([[0, 1], [1, 0]])
R = [1, 2]

In [282]:
mrp = MarkovRewardProcess(gamma=0.9)

In [283]:
mrp.fit(S, P, R)

In [284]:
mrp.value_func()

array([[14.73684211, 15.26315789]])

---

In [30]:
mrp2 = MarkovRewardProcess(alg=ANALYTIC, gamma=0.9)

In [31]:
mrp2.fit(S, P, R)

In [32]:
mrp2.value_func()

[[14.736842105263161], [15.263157894736846]]

---

In [73]:
mrp3 = MarkovRewardProcess(alg=ITERATIVE, gamma=0.9, epsilon=0.001)

In [74]:
mrp3.fit(S, P ,R)

In [75]:
mrp3.value_func()

array([14.73313698, 15.25982328])

---

In [5]:
a = np.random.rand(5, 4)

In [6]:
a

array([[0.08967741, 0.20728536, 0.21037018, 0.37600818],
       [0.33755854, 0.99550407, 0.30595872, 0.83815765],
       [0.8458791 , 0.87183648, 0.2208869 , 0.59762388],
       [0.83535189, 0.55467284, 0.32918539, 0.62810763],
       [0.95835326, 0.24146916, 0.26422433, 0.56791863]])

In [7]:
a[1]

array([0.33755854, 0.99550407, 0.30595872, 0.83815765])

In [8]:
v=np.array([[1],[2],[3],[4]])

In [9]:
np.dot(a[1],v)

array([6.59907344])

# Policy and Value iteration

In [127]:
import numpy as np
from itertools import combinations

In [133]:
POLICY_ITER = "policy_iteration"
VALUE_ITER = "value_iteration"
POLICY_SEARCH = "policy_search" # Not implemented

Mars Rover Env:

(MRP Case):

In [19]:
S = 7
P = np.array([[0.6, 0.4, 0, 0, 0, 0, 0],
             [0.4, 0.2, 0.4, 0, 0, 0, 0],
             [0, 0.4, 0.2, 0.4, 0, 0, 0],
             [0, 0, 0.4, 0.2, 0.4, 0, 0],
             [0, 0, 0, 0.4, 0.2, 0.4, 0],
             [0, 0, 0, 0, 0.4, 0.2, 0.4],
             [0, 0, 0, 0, 0, 0.4, 0.6]])
R = [1, 0, 0, 0, 0, 0, 10]

In [20]:
P

array([[0.6, 0.4, 0. , 0. , 0. , 0. , 0. ],
       [0.4, 0.2, 0.4, 0. , 0. , 0. , 0. ],
       [0. , 0.4, 0.2, 0.4, 0. , 0. , 0. ],
       [0. , 0. , 0.4, 0.2, 0.4, 0. , 0. ],
       [0. , 0. , 0. , 0.4, 0.2, 0.4, 0. ],
       [0. , 0. , 0. , 0. , 0.4, 0.2, 0.4],
       [0. , 0. , 0. , 0. , 0. , 0.4, 0.6]])

In [21]:
mrp_rover = MarkovRewardProcess(gamma=0.5, alg=ANALYTIC)

In [22]:
mrp_rover.fit(S, P, R)

In [178]:
mrp_rover.value_func()

[[1.5342666565342837],
 [0.3699332978699934],
 [0.1304331838806863],
 [0.21701602959309496],
 [0.846138949288241],
 [3.59060924220399],
 [15.311602640629713]]

(MDP Case):

In [298]:
S = 7
A = 2 # left + right
P_non_deterministic = np.array(
            [[[0.9, 0.1, 0, 0, 0, 0, 0],
             [0.9, 0.1, 0, 0, 0, 0, 0],
             [0, 0.9, 0.1, 0, 0, 0, 0],
             [0, 0, 0.9, 0.1, 0, 0, 0],
             [0, 0, 0, 0.9, 0.1, 0, 0],
             [0, 0, 0, 0, 0.9, 0.1, 0],
             [0, 0, 0, 0, 0, 0.9, 0.1]],
             [[0.1, 0.9, 0, 0, 0, 0, 0],
             [0, 0.1, 0.9, 0, 0, 0, 0],
             [0, 0, 0.1, 0.9, 0, 0, 0],
             [0, 0, 0, 0.1, 0.9, 0, 0],
             [0, 0, 0, 0, 0.1, 0.9, 0],
             [0, 0, 0, 0, 0, 0.1, 0.9],
             [0, 0, 0, 0, 0, 0.1, 0.9]]])
R = [1, 0, 0, 0, 0, 0, 10]
GAMMA = 0.99
PI = [1, 1, 1, 0, 0, 0, 0]

In [299]:
class MarkovDecisionProcess():
    def __init__(self, alg=POLICY_ITER, eval_alg=ANALYTIC, gamma=0.99, n_episodes=100, epsilon=0.01):
        self.alg = alg
        self.evaluation_alg = eval_alg
        self.gamma = gamma
        self.n_episodes = n_episodes
        self.epsilon = epsilon
        self.value_function = None
        self._time_steps = 500
        self.best_policy = None
        

    def fit(self, S, P, R, A):
        """
        S: Integer that represent the amount of states
        R: List of rewards at the length of len(S) (reward for each state)
        """
        # Set the value table
        self.value_function = np.zeros(S)
        
        if self.alg == POLICY_ITER: 
            self._fit_policy_iteration(S, P, R, A)
        elif self.alg == VALUE_ITER:
            self._fit_policy_iteration(S, P, R, A)

            
    def _fit_policy_iteration(self, S, P, R, A):
        # Get a random policy
        policy = np.random.randint(A, size=S)
        
        # While policy changes:
        while True:
            # Get policy value
            policy_value = self.policy_evaluation(S, P, R, A, policy)
            
            # Improve the policy
            new_policy = self.policy_improvement(S, P, R, A, policy_value)
            
            # if the policy didn't change, then stop
            if np.array_equal(policy, new_policy):
                break
            else:
                policy = new_policy
        
        # Return this policy and it's value
        self.value_function = policy_value
        self.best_policy = policy
            

    def _fit_value_iteration(self, S, P, R, A):
        policy = []
        self.value_function = np.ones(S) * self.epsilon * 2
        temp_value_function = np.zeros(S)
        
        while np.linalg.norm(self.value_function - temp_value_function) > self.epsilon:
            self.value_function = temp_value_function.copy()
            for state in range(S):
                values = []
                for action in range(A):
                    action_value = R[state] + self.gamma * np.dot(P[action][state], np.transpose(self.value_function))
                    values.append(action_value)
                temp_value_function[state] = np.max(values)
                
        # Get the policy from the best values
        for state in range(S):
            values = []
            for action in range(A):
                action_value = R[state] + self.gamma * np.dot(P[action][state], np.transpose(value_function))
                values.append(action_value)
            policy.append(np.argmax(values))
        
        # Save the policy (We already saved the value of this policy)
        self.best_policy = policy

    def policy_improvement(self, S, P, R, A, value_function):
        policy = []
        for state in range(S):
            values = []
            for action in range(A):
                action_value = R[state] + self.gamma * np.dot(P[action][state], np.transpose(value_function))
                values.append(action_value)
            policy.append(np.argmax(values))
        return np.asarray(policy)
    
            
    def policy_evaluation(self, S, P, R, A, policy):
        transition_matrix = self._build_transition_matrix_from_policy(S, P, policy)
        mrp = MarkovRewardProcess(alg=self.evaluation_alg, gamma=self.gamma, n_episodes=self.n_episodes, epsilon=self.epsilon)
        mrp.fit(S, transition_matrix, R)
        # return the policy value from the evaluation
        return mrp.value_func()
        
    
    def _build_transition_matrix_from_policy(self, S, P, policy):
        transition_matrix = np.zeros((S, S))
        for state, action in zip(range(S), policy):
            transition_matrix[state] = P[action][state]
            
        return transition_matrix

In [300]:
mdp = MarkovDecisionProcess(alg=POLICY_ITER, gamma=GAMMA)

In [301]:
# mdp.policy_evaluation(S, P_non_deterministic, R, A, policy=PI)

In [302]:
mdp.fit(S, P_non_deterministic, R, A)

In [303]:
mdp.best_policy

array([1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [ ]:
mdp.value_function

---

In [ ]:
mdp_iter = MarkovDecisionProcess(alg=VALUE_ITER, gamma=GAMMA)

In [285]:
np.max([1,2,3])

3